# Recipes For Translating A BUGS Model To LogDensityProblems

# `Rats` Model

In [ ]:
# JuliaBUGS
@bugsast begin
    for i in 1:N
        for j in 1:T
            Y[i, j] ~ dnorm(mu[i, j], tau_c)
            mu[i, j] = alpha[i] + beta[i] * (x[j] - xbar)
        end
        alpha[i] ~ dnorm(alpha_c, alpha_tau)
        beta[i] ~ dnorm(beta_c, beta_tau)
    end
    tau_c ~ dgamma(0.001, 0.001)
    sigma = 1 / sqrt(tau_c)
    alpha_c ~ dnorm(0.0, 1.0E-6)
    alpha_tau ~ dgamma(0.001, 0.001)
    beta_c ~ dnorm(0.0, 1.0E-6)
    beta_tau ~ dgamma(0.001, 0.001)
    alpha0 = alpha_c - xbar * beta_c
end

In [1]:
# data
x = [8.0, 15.0, 22.0, 29.0, 36.0]
xbar = 22
N = 30
T = 5
Y = [
    151 199 246 283 320
    145 199 249 293 354
    147 214 263 312 328
    155 200 237 272 297
    135 188 230 280 323
    159 210 252 298 331
    141 189 231 275 305
    159 201 248 297 338
    177 236 285 350 376
    134 182 220 260 296
    160 208 261 313 352
    143 188 220 273 314
    154 200 244 289 325
    171 221 270 326 358
    163 216 242 281 312
    160 207 248 288 324
    142 187 234 280 316
    156 203 243 283 317
    157 212 259 307 336
    152 203 246 286 321
    154 205 253 298 334
    139 190 225 267 302
    146 191 229 272 302
    157 211 250 285 323
    132 185 237 286 331
    160 207 257 303 345
    169 216 261 295 333
    157 205 248 289 316
    137 180 219 258 291
    153 200 244 286 324
]

# parameters
alpha = [250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250]
beta = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
alpha_c = 150
beta_c = 10
tau_c = 1
alpha_tau = 1
beta_tau = 1;

# With [DynamicHMC.jl](https://github.com/tpapp/DynamicHMC.jl)

In [4]:
using LogDensityProblems
using UnPack, BangBang
using Distributions
using StaticArrays

# also defined in JuliaBUGS.jl
dnorm(μ, τ) = Normal(μ, 1 / sqrt(τ))
dgamma(a, b) = Gamma(a, 1 / b) 

struct Rats
    x
    xbar
    Y
    N::Int
    T::Int
end

function (problem::Rats)(vars)
    @unpack x, xbar, Y, N, T = problem
    @unpack alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau = vars

    # to work with AutoDiff, mutation is generally not permitted
    # in this example, we can just directly plug in the definition of `mu[i, j]` into the location it's used
    
    mu = ones(N, T)
    for i in 1:N
        for j in 1:T
            setindex!!(mu, alpha[i] + beta[i] * (x[j] - xbar), i, j)
        end
    end
    
    loglikelihood = 0.0
    for i in 1:N
        for j in 1:T
            loglikelihood += logpdf(dnorm(mu[i, j], tau_c), Y[i, j])
        end
    end
    for i in 1:N
        loglikelihood += logpdf(dnorm(alpha_c, alpha_tau), alpha[i])
        loglikelihood += logpdf(dnorm(beta_c, beta_tau), beta[i])
    end

    loglikelihood += logpdf(dgamma(0.001, 0.001), tau_c)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), alpha_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), alpha_tau)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), beta_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), beta_tau)
    return loglikelihood
end

function LogDensityProblems.logdensity(p::Rats, vars)
    return p(vars)
end

function LogDensityProblems.capabilities(::Type{<:Rats})
    return LogDensityProblems.LogDensityOrder{1}()
end

function LogDensityProblems.dimension(p::Rats)
    @unpack x, xbar, Y, N, T = p
    return N + N + 1 + 1 + 1 + 1 + 1
end

In [5]:
p = Rats(x, xbar, Y, N, T)
p((alpha = alpha, beta = beta, tau_c = tau_c, alpha_c = alpha_c, alpha_tau = alpha_tau, beta_c = beta_c, beta_tau = beta_tau))

-174029.38703951906

In [ ]:
using TransformVariables, TransformedLogDensities
t = as((alpha = as(Array, N), beta = as(Array, N), tau_c = asℝ₊, alpha_c = asℝ, alpha_tau = asℝ₊, beta_c = asℝ, beta_tau = asℝ₊))
t_p = TransformedLogDensity(t, p)

In [ ]:
using LogDensityProblemsAD, DynamicHMC, Random, ForwardDiff
∇P = ADgradient(:ForwardDiff, t_p)

In [ ]:
inits = (alpha = alpha, beta = beta, tau_c = tau_c, alpha_c = alpha_c, alpha_tau = alpha_tau, beta_c = beta_c, beta_tau = beta_tau)
results = mcmc_with_warmup(Random.GLOBAL_RNG, ∇P, 1000)

In [ ]:
# check the posterior of variable `beta_c`
transformed_results = transform.(t, eachcol(results.posterior_matrix))
beta_c_samples = [transformed_results[s][:beta_c] for s in 1:length(transformed_results)]
stats = mean(beta_c_samples), std(beta_c_samples) # reference results: mean 6.186, variance 0.1088

# With [AdvancedHMC.jl](https://github.com/TuringLang/AdvancedHMC.jl)

HMC needs parameters to be unconfined, the second argument of function `dnorm` is required to be positive. We can use the package [`Bijectors.jl`](https://github.com/TuringLang/Bijectors.jl).

In [ ]:
using Bijectors
b = Bijectors.Log{0}()
b⁻¹ = Bijectors.Exp{0}();

In [ ]:
using Distributions

In [ ]:
d = Gamma(1, 1)

In [ ]:
b = bijector(d)

In [ ]:
Bijectors.inverse(b)

In [ ]:
tau_c = b(tau_c) # 1 => 0
alpha_tau = b(alpha_tau) # 1 => 0
beta_tau = b(beta_tau); # 1 => 0

In [ ]:
struct Rats
    x
    xbar
    Y
    N::Int
    T::Int
end

function (problem::Rats)(vars)
    @unpack x, xbar, Y, N, T = problem
    alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau = vars[1:N], vars[N+1:2N], vars[2N+1:end]... # parameters are linearized

    b⁻¹ = Bijectors.Exp{0}()
    tau_c = b⁻¹(tau_c)
    alpha_tau = b⁻¹(alpha_tau)
    beta_tau = b⁻¹(beta_tau)
    
    loglikelihood = 0.0
    for i in 1:N
        for j in 1:T
            loglikelihood += logpdf(dnorm(alpha[i] + beta[i] * (x[j] - xbar), tau_c), Y[i, j])
        end
    end
    for i in 1:N
        loglikelihood += logpdf(dnorm(alpha_c, alpha_tau), alpha[i])
        loglikelihood += logpdf(dnorm(beta_c, beta_tau), beta[i])
    end

    loglikelihood += logpdf(dgamma(0.001, 0.001), tau_c)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), alpha_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), alpha_tau)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), beta_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), beta_tau)
    return loglikelihood
end

In [ ]:
p = Rats(x, xbar, Y, N, T)
initial_θ = convert(Array{Float64}, vcat(alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau))
p(initial_θ)

In [ ]:
using AdvancedHMC, ForwardDiff

D = length(initial_θ)
n_samples, n_adapts = 2000, 1000

metric = DiagEuclideanMetric(D)
hamiltonian = Hamiltonian(metric, p, :ForwardDiff)

initial_ϵ = find_good_stepsize(hamiltonian, initial_θ)
integrator = Leapfrog(initial_ϵ)
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; drop_warmup=true, progress=true)

In [ ]:
beta_c_samples = [samples[s][64] for s in 1:length(samples)]
stats = mean(beta_c_samples), std(beta_c_samples) # Reference result: mean 6.186, variance 0.1088

In [ ]:
tau_c_samples = [samples[s][61] for s in 1:length(samples)]
sigma_samples = map(x->1/sqrt(b⁻¹(x)), tau_c_samples)
stats = mean(sigma_samples), std(sigma_samples) # Reference result: mean 6.092, sd 0.4672